In [4]:
import requests
import time
from bs4 import BeautifulSoup
import pandas as pd

In [5]:
def numStr(num):
    if num >= 10:
        return str(num)
    else:
        return '0' + str(num)

In [ ]:
Base = "https://race.netkeiba.com/?pid=race_old&id=c"
dst = ''
df_col = [
    'year', 'date', 'field', 'race', 'race_name'
          , 'course', 'head_count', 'rank', 'horse_name'
          , 'gender', 'age', 'trainerA', 'trainerB', 'weight', 'c_weight', 'jackie', 'j_weight'
          , 'odds','popu']
df = pd.DataFrame()